<a href="https://colab.research.google.com/github/Omo-Tines/Sentiment-Analysis-ML/blob/main/SENIMLLAB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U torch==0.8.0 torchtext==0.9.0
#==1.11.0
#==0.9.0

# Reload environment
exit()

ERROR: Could not find a version that satisfies the requirement torch==0.8.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch==0.8.0


In [1]:
#Creating the tokenizer and labels
import torch
import torchtext
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = torchtext.data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

#Importing the datasets

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.0MB/s]


#Splitting the datasets

In [4]:
import random
train_data, valid_data = train_data.split(
    split_ratio=[0.75, 0.25],
    random_state=random.seed(SEED))
print(len(train_data))
print(len(valid_data))

18750
6250


In [5]:
test_data, other_data = test_data.split(
    split_ratio=[0.75, 0.25],
    random_state=random.seed(SEED))

#Building a vocabulary

In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:20<00:00, 19123.45it/s]


In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)
test_loader=train_iterator

In [8]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):

        #text = [sent len, batch size]

        embedded = self.dropout(self.embedding(text))

        #embedded = [sent len, batch size, emb dim]

        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))

        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors

        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]

        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        #hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [11]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [12]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1071,  0.6057,  0.6343,  ...,  0.3833,  0.6149,  0.4702],
        [-1.0149,  1.9495, -1.0922,  ..., -0.8807, -0.4166, -0.8111],
        [-1.0410, -0.0363,  1.5732,  ..., -0.9128,  0.9207, -2.4707]])

In [13]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1071,  0.6057,  0.6343,  ...,  0.3833,  0.6149,  0.4702],
        [-1.0149,  1.9495, -1.0922,  ..., -0.8807, -0.4166, -0.8111],
        [-1.0410, -0.0363,  1.5732,  ..., -0.9128,  0.9207, -2.4707]])


In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [15]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [16]:
def binary_accuracy(preds, y):
  """
  Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
  """
  #round predictions to the closest integer
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float() #convert into float for division
  acc = correct.sum() / len(correct)
  return acc

In [17]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [18]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            txt, text_lengths = batch.text

            predictions = model(txt, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [19]:
import time

def epoch_time(start_time, end_time):
    elps_time = end_time - start_time
    elps_mins = int(elps_time / 60)
    elps_secs = int(elps_time - (elps_mins * 60))
    return elps_mins, elps_secs

In [20]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [21]:
numepo = 5

best_val_loss = float('inf')

for epoch in range(numepo):

    start_time = time.time()

    tr_loss, tr_acc = train(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = evaluate(model, valid_iterator, criterion)

    endt = time.time()

    emins, esecs = epoch_time(start_time, endt)

    if val_loss < best_val_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {emins}m {esecs}s')
    print(f'\tTrain Loss: {tr_loss:.3f} | Train Acc: {tr_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 34s
	Train Loss: 0.651 | Train Acc: 60.89%
	 Val. Loss: 0.673 |  Val. Acc: 58.99%
Epoch: 02 | Epoch Time: 0m 35s
	Train Loss: 0.636 | Train Acc: 63.45%
	 Val. Loss: 0.572 |  Val. Acc: 71.19%
Epoch: 03 | Epoch Time: 0m 38s
	Train Loss: 0.540 | Train Acc: 72.76%
	 Val. Loss: 0.372 |  Val. Acc: 84.66%
Epoch: 04 | Epoch Time: 0m 38s
	Train Loss: 0.475 | Train Acc: 78.18%
	 Val. Loss: 0.538 |  Val. Acc: 74.70%
Epoch: 05 | Epoch Time: 0m 37s
	Train Loss: 0.369 | Train Acc: 84.26%
	 Val. Loss: 0.369 |  Val. Acc: 86.13%


In [23]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.367 | Test Acc: 85.57%


In [22]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    index = [TEXT.vocab.stoi[t] for t in tokenized]
    lgth = [len(index)]
    tensor = torch.LongTensor(index).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(lgth)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [24]:
predict_sentiment(model, "This film is terrible")
#close to 0 is negative and close to 1 is positive

0.020738571882247925

In [25]:
predict_sentiment(model, "This film is amazing")
#close to 0 is negative and close to 1 is positive

0.9962856769561768

In [41]:
#hide
import torchtext
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.text.all import *
path = untar_data(URLs.YELP_REVIEWS)

ImportError: ignored

In [ ]:
#ylpdata= datasets.YelpReviewFull

In [38]:
Path.BASE_PATH = path
path.ls()

NameError: ignored

In [ ]:

YLPTEXT = torchtext.legacy.data.LabelField(dtype=torch.long)
### Defining the label processing

YLPLABEL = torchtext.legacy.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)


In [ ]:
fields = [('TEXT_COLUMN_NAME',YLPTEXT ), ('LABEL_COLUMN_NAME', YLPLABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path=path/'test.csv', format='csv',
    skip_header=True, fields=fields)

In [ ]:
import torch
import torch.nn.functional as F
ylptest_data, no_test = dataset.split(
    split_ratio=[0.10, 0.90],
    random_state=random.seed(SEED))

print(f'Num Test: {len(ylptest_data)}')
print(f'Num non Test: {len(no_test)}')

In [ ]:
# trdataset = torchtext.legacy.data.TabularDataset(
#     path=path/'train.csv', format='csv',
#     skip_header=True, fields=fields)

In [ ]:
# trainylp_data, validylp_data = dataset.split(
#     split_ratio=[0.8, 0.2],
#     random_state=random.seed(SEED))

# print(f'Num Train: {len(trainylp_data)}')
# print(f'Num Valid: {len(validylp_data)}')

In [ ]:
print(vars(ylptest_data.examples[0]))

In [ ]:
YLPLABEL.build_vocab(ylptest_data, max_size=MAX_VOCAB_SIZE)
YLPTEXT.build_vocab(ylptest_data)

print(f'Vocabulary size: {len(YLPLABEL.vocab)}')
print(f'Number of classes: {len(YLPTEXT.vocab)}')

In [ ]:
print(YLPTEXT.vocab.stoi)

In [ ]:
yelp_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tst_loader = \
    torchtext.legacy.data.BucketIterator(
        ylptest_data,
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.LABEL_TEXT_NAME),
         device=yelp_device
    )

In [ ]:
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.LABEL_COLUMN_NAME.size()}')

    break

In [ ]:
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

In [ ]:
#tloss, tacc=train(model, test_loader, optimizer, criterion)
tloss, tacc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {tloss:.3f} | Test Acc: {tacc*100:.2f}%')

In [ ]:
path1 = untar_data(URLs.IMDB)
Path.BASE_PATH = path1
path1.ls()

In [ ]:
negl1=L()
negl2=L()
negl3=L()
negl4=L()
negl5=L()
posl1=L()
posl2=L()
posl3=L()
posl4=L()
posl5=L()
with open(path1/'test/neg/14_1.txt') as f:negl1 += L(*f.readlines())
with open(path1/'test/neg/15_2.txt') as f:negl2+= L(*f.readlines())
with open(path1/'test/neg/16_1.txt') as f:negl3+= L(*f.readlines())
with open(path1/'test/neg/17_3.txt') as f:negl4+= L(*f.readlines())
with open(path1/'test/neg/18_1.txt') as f:negl5+= L(*f.readlines())
with open(path1/'test/pos/13_9.txt') as f:posl1 += L(*f.readlines())
with open(path1/'test/pos/14_8.txt') as f:posl2+= L(*f.readlines())
with open(path1/'test/pos/16_8.txt') as f:posl3+= L(*f.readlines())
with open(path1/'test/pos/17_8.txt') as f:posl4+= L(*f.readlines())
with open(path1/'test/pos/18_8.txt') as f:posl5+= L(*f.readlines())


In [ ]:
predict_sentiment(model,str(negl1)),negl1

In [ ]:
predict_sentiment(model,str(negl2)),negl2

In [ ]:
predict_sentiment(model,str(negl3)),negl3

In [ ]:
predict_sentiment(model,str(negl4)),negl4

In [ ]:
predict_sentiment(model,str(negl5)),negl5

In [ ]:
predict_sentiment(model,str(posl1)),posl1

In [ ]:
predict_sentiment(model,str(posl2)),posl2

In [ ]:
predict_sentiment(model,str(posl3)),posl3

In [ ]:
predict_sentiment(model,str(posl4)),posl4

In [ ]:
predict_sentiment(model,str(posl5)),posl5

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[0]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[1]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[2]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[3]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[4]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[5]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[6]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[7]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[8]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[9]).values()))

In [ ]:
predict_sentiment(model,str(vars(ylptest_data.examples[10]).values()))